In [2]:
!pip install requests
!pip install pandas
!pip install numpy
!pip install seaborn 
!pip install zipfile

In [3]:
import requests
import pandas as pd
import seaborn as sns 
import numpy as np
import zipfile as zp

In [4]:

# Path to the original zip file
original_zip_path = 'data/NADAC-2024.csv.zip'

# Extract the CSV file from the zip archive and load it into a DataFrame
with zp.ZipFile(original_zip_path, 'r') as zipf:
    with zipf.open('NADAC-2024.csv') as file:
        df_nadac = pd.read_csv(file)

# Split the DataFrame into two halves
df_nadac_part1 = df_nadac.iloc[:len(df_nadac) // 2]
df_nadac_part2 = df_nadac.iloc[len(df_nadac) // 2:]

# Save each part to a separate zip file
with zp.ZipFile('data/NADAC-2024_part1.csv.zip', 'w') as zipf:
    zipf.writestr('NADAC-2024_part1.csv', df_nadac_part1.to_csv(index=False))

with zp.ZipFile('data/NADAC-2024_part2.csv.zip', 'w') as zipf:
    zipf.writestr('NADAC-2024_part2.csv', df_nadac_part2.to_csv(index=False))

print("Split and saved NADAC-2024.csv into two zip files: NADAC-2024_part1.csv.zip and NADAC-2024_part2.csv.zip")


Split and saved NADAC-2024.csv into two zip files: NADAC-2024_part1.csv.zip and NADAC-2024_part2.csv.zip


READING IN DATASETS 

In [5]:
import pandas as pd
import zipfile as zp

# List of zip files, including the split NADAC files
zip_files = [
    'data/CMSspending.csv.zip',
    'data/fda_directory.xlsx.zip',
    'data/NADAC-2024_part1.csv.zip',
    'data/NADAC-2024_part2.csv.zip',
]

# Create an empty dictionary to store each DataFrame
dataframes = {}

# Temporary list to hold NADAC parts
nadac_parts = []

# Loop through each zip file and extract the content
for zip_path in zip_files:
    with zp.ZipFile(zip_path, 'r') as zipf:
        # Get the name of the first file inside the zip (assuming only one file per zip)
        file_name = zipf.namelist()[0]
        
        # Extract the file and load it into a DataFrame based on its file extension
        with zipf.open(file_name) as file:
            if file_name.endswith('.csv'):
                # Load CSV file into DataFrame
                df = pd.read_csv(file)
                # If the file is a part of NADAC, add it to the nadac_parts list
                if 'NADAC-2024_part' in file_name:
                    nadac_parts.append(df)
                else:
                    # Store other CSV files normally
                    base_name = file_name.split('.')[0]
                    dataframes[base_name] = df
            elif file_name.endswith('.xlsx'):
                # Load Excel file into DataFrame
                df = pd.read_excel(file, engine='openpyxl')
                # Store the Excel file normally
                base_name = file_name.split('.')[0]
                dataframes[base_name] = df
            else:
                raise ValueError(f"Unsupported file format: {file_name}")

# Concatenate the NADAC parts into a single DataFrame and add it to the dictionary
if nadac_parts:
    dataframes['NADAC-2024'] = pd.concat(nadac_parts, ignore_index=True)
    print("Merged NADAC parts into a single DataFrame")

# Display the first few rows of each DataFrame for verification
for name, df in dataframes.items():
    print(f"\nDataFrame '{name}':")
    print(df.head())


Merged NADAC parts into a single DataFrame

DataFrame 'CMSspending':
  Utilization Type State      NDC  Labeler Code  Product Code  Package Size  \
0             MCOU    XX  2010102             2           101             2   
1             FFSU    XX  2021301             2           213             1   
2             MCOU    XX  2021301             2           213             1   
3             MCOU    XX  2035102             2           351             2   
4             FFSU    XX  2091155             2           911            55   

   Year  Quarter  Suppression Used Product Name  Units Reimbursed  \
0  2024        2             False   AMMONIUM C            21.000   
1  2024        2             False   HUMULIN R           4106.000   
2  2024        2             False   HUMULIN R           2465.532   
3  2024        2              True   DARVOCET-N               NaN   
4  2024        2              True                            NaN   

   Number of Prescriptions  Total Amount 

In [6]:
#Create dataframes for each dataset 

df_directory = dataframes["fda_directory"]
df_spending = dataframes["CMSspending"]
df_pricing = dataframes["NADAC-2024"]
df_purplebook = pd.read_csv("data/purplebook-october-data.csv")
df_pricing

,NDC Description,NDC,NADAC Per Unit,Effective Date,Pricing Unit,Pharmacy Type Indicator,OTC,Explanation Code,Classification for Rate Setting,Corresponding Generic Drug NADAC Per Unit,Corresponding Generic Drug Effective Date,As of Date
0,12HR NASAL DECONGEST ER 120 MG,24385005452,0.26838,12/20/2023,EA,C/I,Y,1,G,NaN,NaN,01/03/2024
1,12HR NASAL DECONGEST ER 120 MG,70000047501,0.26838,12/20/2023,EA,C/I,Y,1,G,NaN,NaN,01/03/2024
2,24H NASAL ALLERGY 55 MCG SPRAY,46122038576,0.76188,12/20/2023,ML,C/I,Y,"1, 5",G,NaN,NaN,01/03/2024
3,24HR ALLERGY(LEVOCETIRZN) 5 MG,70000036201,0.20773,12/20/2023,EA,C/I,Y,1,G,NaN,NaN,01/03/2024
4,24HR ALLERGY(LEVOCETIRZN) 5 MG,70000036202,0.20773,12/20/2023,EA,C/I,Y,1,G,NaN,NaN,01/03/2024
...,...,...,...,...,...,...,...,...,...,...,...,...
1288426,ZYPREXA 20 MG TABLET,61269063530,49.79369,10/16/2024,EA,C/I,N,2,B,0.15483,10/23/2024,11/06/2024
1288427,ZYPREXA 5 MG TABLET,2411530,15.24447,09/18/2024,EA,C/I,N,2,B,0.08490,10/23/2024,11/06/2024
1288428,ZYPREXA 7.5 MG TABLET,2411630,18.26917,09/18/2024,EA,C/I,N,2,B,0.10835,10/23/2024,11/06/2024
1288429,ZYPREXA ZYDIS 5 MG TABLET,2445301,16.11767,09/18/2024,EA,C/I,N,4,B,0.36541,10/23/2024,11/06/2024


CLEANING AND MERGING DATA

EXTRANEOUS

In [3]:
url = "https://data.medicaid.gov/api/1/datastore/query/99315a95-37ac-4eee-946a-3c523b4c481e/0"

In [ ]:
database = []
offset = 0
while True:
    response = requests.get(url, params={'limit': 5000, 'offest': offset, 'format': 'json'})
    data = response.json().get('results', [])
    if not data:
        break
    print('iteration')
    database.extend(data)
    offset += 5000

medicaid_df = pd.DataFrame(database)